In [1]:
import sqlite3
from sqlite3 import Error

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

In [3]:
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
SELECT name AS facility_name, 
    revenue
FROM (
    SELECT 
        name,
        SUM(CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
            ELSE f.membercost * b.slots END
            ) as revenue
        FROM Bookings AS b
        LEFT JOIN Members AS m
            ON b.memid = m.memid
        LEFT JOIN Facilities AS f
            ON b.facid = f.facid
        GROUP BY name
        ) AS booking
WHERE revenue < 1000
ORDER BY revenue DESC;
        """
    query2 = """
SELECT (m1.surname || ", " || m1.firstname) AS Member_Name,
    (m2.surname || ", " || m2.firstname) AS Recommended_By
FROM Members as m1
INNER JOIN Members as m2
ON m1.recommendedby = m2.memid
WHERE m2.surname <> 'GUEST'
ORDER BY Member_Name, Recommended_BY;   
    """
    
    query3 = """
SELECT name AS facility, count(memid) AS facility_usage
FROM Bookings as b
INNER JOIN Facilities as f
USING (facid)
WHERE b.memid <> 0
GROUP BY b.facid;
    """
    
    query4 = """
SELECT name AS facility,
    strftime('%m',starttime) as month,
    COUNT(memid) AS monthly_usage
FROM Bookings AS b
INNER JOIN Facilities as f
USING (facid)
WHERE memid <> 0
GROUP BY facid, month;  
    """
    
    queries = (query1, query2, query3, query4)

    for query in queries:
        cur.execute(query)
 
        rows = cur.fetchall()
 
        for row in rows:
            print(row)
        
        print('\n')

In [4]:
def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)

In [5]:
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Pool Table', 270)
('Snooker Table', 240)
('Table Tennis', 180)


('Bader, Florence', 'Stibbons, Ponder')
('Baker, Anne', 'Stibbons, Ponder')
('Baker, Timothy', 'Farrell, Jemima')
('Boothe, Tim', 'Rownam, Tim')
('Butters, Gerald', 'Smith, Darren')
('Coplin, Joan', 'Baker, Timothy')
('Crumpet, Erica', 'Smith, Tracy')
('Dare, Nancy', 'Joplette, Janice')
('Genting, Matthew', 'Butters, Gerald')
('Hunt, John', 'Purview, Millicent')
('Jones, David', 'Joplette, Janice')
('Jones, Douglas', 'Jones, David')
('Joplette, Janice', 'Smith, Darren')
('Mackenzie, Anna', 'Smith, Darren')
('Owen, Charles', 'Smith, Darren')
('Pinker, David', 'Farrell, Jemima')
('Purview, Millicent', 'Smith, Tracy')
('Rumney, Henrietta', 'Genting, Matthew')
('Sarwin, Ramnaresh', 'Bader, Florence')
('Smith, Jack', 'Smith, Darren')
('Stibbons, Ponder', 'Tracy, Burton')
('Worthington-Smyth, Henry', 'Smith, Tracy')


('Tennis Court 1', 308)
('Tennis Court 2', 276)
('Badminton Court', 344)
('Table Ten